## 基于截断

In [17]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, Trainer, TrainingArguments, DefaultDataCollator

## 数据集加载

In [4]:
datasets = load_dataset("cmrc2018",cache_dir= "./cmrc")
datasets["train"][:10]

{'id': ['TRAIN_186_QUERY_0',
  'TRAIN_186_QUERY_1',
  'TRAIN_186_QUERY_2',
  'TRAIN_186_QUERY_3',
  'TRAIN_186_QUERY_4',
  'TRAIN_54_QUERY_0',
  'TRAIN_54_QUERY_1',
  'TRAIN_54_QUERY_2',
  'TRAIN_54_QUERY_3',
  'TRAIN_756_QUERY_0'],
 'context': ['范廷颂枢机（，），圣名保禄·若瑟（），是越南罗马天主教枢机。1963年被任为主教；1990年被擢升为天主教河内总教区宗座署理；1994年被擢升为总主教，同年年底被擢升为枢机；2009年2月离世。范廷颂于1919年6月15日在越南宁平省天主教发艳教区出生；童年时接受良好教育后，被一位越南神父带到河内继续其学业。范廷颂于1940年在河内大修道院完成神学学业。范廷颂于1949年6月6日在河内的主教座堂晋铎；及后被派到圣女小德兰孤儿院服务。1950年代，范廷颂在河内堂区创建移民接待中心以收容到河内避战的难民。1954年，法越战争结束，越南民主共和国建都河内，当时很多天主教神职人员逃至越南的南方，但范廷颂仍然留在河内。翌年管理圣若望小修院；惟在1960年因捍卫修院的自由、自治及拒绝政府在修院设政治课的要求而被捕。1963年4月5日，教宗任命范廷颂为天主教北宁教区主教，同年8月15日就任；其牧铭为「我信天主的爱」。由于范廷颂被越南政府软禁差不多30年，因此他无法到所属堂区进行牧灵工作而专注研读等工作。范廷颂除了面对战争、贫困、被当局迫害天主教会等问题外，也秘密恢复修院、创建女修会团体等。1990年，教宗若望保禄二世在同年6月18日擢升范廷颂为天主教河内总教区宗座署理以填补该教区总主教的空缺。1994年3月23日，范廷颂被教宗若望保禄二世擢升为天主教河内总教区总主教并兼天主教谅山教区宗座署理；同年11月26日，若望保禄二世擢升范廷颂为枢机。范廷颂在1995年至2001年期间出任天主教越南主教团主席。2003年4月26日，教宗若望保禄二世任命天主教谅山教区兼天主教高平教区吴光杰主教为天主教河内总教区署理主教；及至2005年2月19日，范廷颂因获批辞去总主教职务而荣休；吴光杰同日真除天主教河内总教区

## 数据预处理

In [3]:
tokenizer = AutoTokenizer.from_pretrained("hfl/chinese-macbert-base")
tokenizer

BertTokenizerFast(name_or_path='hfl/chinese-macbert-base', vocab_size=21128, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [7]:
def process_func(example):
    tokenized_example = tokenizer(
        text = example["question"],
        text_pair = example["context"],
        padding = "max_length",
        return_offsets_mapping = True,
        max_length = 512,
        truncation = "only_second"
    )
    offsets_mapping = tokenized_example.pop("offset_mapping")
    start_position = []
    end_position = []
    
    for idx,offset in enumerate(offsets_mapping):
        answer = example["answers"][idx]
        start_char = answer["answer_start"][0]
        end_char = start_char + len(answer["text"][0])
        
        context_start = tokenized_example.sequence_ids(idx).index(1)
        context_end = tokenized_example.sequence_ids(idx).index(None, context_start) -1
        
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char :
            start_token_pos = 0
            end_token_pos = 0
        else:
            id = context_start
            while id <= context_end and offset[id][0] < start_char:
                id += 1
            start_token_pos = id
            id = context_end
            while id >= context_start and offset[id][1] > end_char:
                id -= 1
            end_token_pos = id
        
        start_position.append(start_token_pos)
        end_position.append(end_token_pos)
        
    tokenized_example["start_positions"] = start_position
    tokenized_example["end_positions"] = end_position
    
    return tokenized_example
            

In [8]:
tokenized_datasets = datasets.map(process_func,batched= True, remove_columns= datasets["train"].column_names)
tokenized_datasets

Map: 100%|██████████| 1002/1002 [00:00<00:00, 2375.92 examples/s]


DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 10142
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 3219
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 1002
    })
})

In [9]:
model = AutoModelForQuestionAnswering.from_pretrained("hfl/chinese-macbert-base")


Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at hfl/chinese-macbert-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
args = TrainingArguments(
    output_dir= "model_for_qa",
    per_device_train_batch_size= 32,
    per_device_eval_batch_size= 32,
    eval_strategy= "epoch",
    save_strategy= "epoch",
    logging_steps= 50,
    num_train_epochs=1
)

In [18]:
trainer = Trainer(
    model= model,
    args= args,
    train_dataset= tokenized_datasets["train"],
    eval_dataset= tokenized_datasets["validation"],
    data_collator= DefaultDataCollator()
)

In [20]:
trainer.train()

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [21]:
from transformers import pipeline

pipe = pipeline("question-answering", model = model , tokenizer= tokenizer)

pipe

Device set to use cpu


In [22]:
pipe(question= "小明在哪里上班？", context= "小明在北京上班。")

{'score': 0.04801243916153908, 'start': 3, 'end': 7, 'answer': '北京上班'}

## 训练太久了懒得训练了，主要是数据处理，采用截断的方法，将context截断，然后双指针向答案逼近，如果答案被截断就将答案设为CLS，用两输出头训练起始位置和结束位置。offset里面是每个token对应的字符位置，而start_char和end_char为相应字符，需判断是否超出token范围，向答案逼近